# Tensorflow predictive model on public Google Analytics
#### import relevant modules and packages

In [1]:
# packages
import os

# local modules
from bigquery_interface import CREDENTIAL_FILE_PATH
from bigquery_interface import build_bigquery_client_with_default_dataset
from bigquery_interface import send_query_to_database

The first step to run this project is to initialize a python api that allows me to interface with bigquery. This is done with the functions in build_bigquery_client. For further reference, I also find it conveniente to set the google analytics dataset as defeault dataset for successive queries, and to get its information schema. Getting the information schema allows also to check that the interface API has been properly set up.

In [2]:
# initialize client with default dataset
default_dataset = 'bigquery-public-data.google_analytics_sample'    
client, config_object_default_dataset = build_bigquery_client_with_default_dataset(default_dataset)   

# run query to get processed information about the dataset schema
schema_query_path = os.path.join('query_folder', 'schema_query.sql')
dataset_schema = send_query_to_database(client, 
                                        config_object_default_dataset,
                                        schema_query_path)

# print dataset schema
dataset_schema

,filed_name,tables_with_field
0,visitorId,366
1,visitNumber,366
2,visitId,366
3,visitStartTime,366
4,date,366
5,totals,366
6,trafficSource,366
7,device,366
8,geoNetwork,366
9,customDimensions,366


This summary tells that every all the dataset is made by 366 tables which all share the same columns. This is expected because the dataset has one table for each day of the year 2016. Note that clientId is an anomaly, as it appears only in a subset of the tables. For this reason, I will ignore the column in further analysis.

The schema od the dataset seems quite simple, so I can now investigate the dataset a bit more in detail. Let's start by fetching a snippet of the dataset and see what kind of info I can get out of it

In [6]:
# run query to get snippet of dataset
snippet_query_path = os.path.join('query_folder', 'snippet_query.sql')
snippet = send_query_to_database(client, 
                                 config_object_default_dataset,
                                 snippet_query_path)
snippet

,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,hits,fullVisitorId,userId,clientId,channelGrouping,socialEngagementType
0,NaN,13,1480397622,1480397622,20161128,"{'visits': 1, 'hits': 5, 'pageviews': 5, 'time...","{'referralPath': '/watch', 'campaign': '(not s...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 21, 'minu...",6837748114500171233,None,None,Social,Not Socially Engaged
1,NaN,1,1480335543,1480335543,20161128,"{'visits': 1, 'hits': 6, 'pageviews': 3, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Safari', 'browserVersion': 'not a...","{'continent': 'Asia', 'subContinent': 'Souther...","[{'index': 4, 'value': 'APAC'}]","[{'hitNumber': 1, 'time': 0, 'hour': 4, 'minut...",0562216437867751266,None,None,Organic Search,Not Socially Engaged
2,NaN,1,1480338108,1480338108,20161128,"{'visits': 1, 'hits': 9, 'pageviews': 9, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Edge', 'browserVersion': 'not ava...","{'continent': 'Europe', 'subContinent': 'Weste...","[{'index': 4, 'value': 'EMEA'}]","[{'hitNumber': 1, 'time': 0, 'hour': 5, 'minut...",8345017619107269568,None,None,Direct,Not Socially Engaged
3,NaN,1,1480345768,1480345768,20161128,"{'visits': 1, 'hits': 12, 'pageviews': 10, 'ti...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Safari', 'browserVersion': 'not a...","{'continent': 'Europe', 'subContinent': 'Weste...","[{'index': 4, 'value': 'EMEA'}]","[{'hitNumber': 1, 'time': 0, 'hour': 7, 'minut...",2035721900226604057,None,None,Organic Search,Not Socially Engaged
4,NaN,3,1480346206,1480346206,20161128,"{'visits': 1, 'hits': 8, 'pageviews': 8, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Europe', 'subContinent': 'Weste...","[{'index': 4, 'value': 'EMEA'}]","[{'hitNumber': 1, 'time': 0, 'hour': 7, 'minut...",5351678518557340681,None,None,Direct,Not Socially Engaged


The snippet indicates that the data are highly nested in this dataset. Moreover, information about time and dates have a non-standard format (Ouch). Some of the columns seems to have also a bunch of obfuscated values. This is consistent with the open info from the dataset. Let's figure out with an extra query.

In [14]:
# run query to investigate which columns can be discarded from the analysis
var_info_query_path = os.path.join('query_folder', 'variables_of_interest_query.sql')
var_info = send_query_to_database(client, 
                                  config_object_default_dataset,
                                  var_study_query_path)
var_info

BadRequest: 400 Aggregate functions with DISTINCT cannot be used with arguments of type STRUCT at [7:5]

(job ID: 2423a6cd-a891-43ac-912c-600ccfe39d32)

                 -----Query Job SQL Follows-----                 

    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:SELECT 
   2:    COUNT(DISTINCT visitorId)      AS unique_visitorID,
   3:    COUNT(DISTINCT visitNumber)    AS unique_visitNumber,
   4:    COUNT(DISTINCT visitId)        AS unique_visitId,
   5:    COUNT(DISTINCT visitStartTime) AS unique_visitStartTime,
   6:    COUNT(DISTINCT date)           AS unique_date,
   7:    COUNT(DISTINCT totals)         AS unique_totals,
   8:    COUNT(DISTINCT trafficSource)  AS unique_trafficSource,
   9:    COUNT(DISTINCT device)         AS unique_device
  10:FROM `ga_sessions_*`
  11:LIMIT 5;    
    |    .    |    .    |    .    |    .    |    .    |    .    |

Which tells us that a few variables are completely obfuscated, such as
1. visitorId
so I can drop them to limit bandwith consumption